In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Path to chromedriver
!which chromedriver

which: no chromedriver in (/c/Users/emily/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/emily/bin:/c/Python39/Scripts:/c/Python39:/c/Windows/system32:/c/Windows:/c/Windows/System32/Wbem:/c/Windows/System32/WindowsPowerShell/v1.0:/c/Windows/System32/OpenSSH:/cmd:/c/Program Files/MongoDB/Server/4.4/bin:/c/Program Files/nodejs:/c/ProgramData/chocolatey/bin:/c/Users/emily/anaconda3:/c/Users/emily/anaconda3/Library/mingw-w64/bin:/c/Users/emily/anaconda3/Library/usr/bin:/c/Users/emily/anaconda3/Library/bin:/c/Users/emily/anaconda3/Scripts:/c/Users/emily/AppData/Local/Microsoft/WindowsApps:/c/Users/emily/AppData/Local/Programs/Microsoft VS Code/bin:/c/Users/emily/AppData/Roaming/npm:/usr/bin/vendor_perl:/usr/bin/core_perl)


In [3]:
# Set the executable path and initialize the chrome browser in splinter
# executable_path = {'executable_path': 'chromedriver.exe'}
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\emily\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [ ]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

In [ ]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('ul.item_list li.slide')

In [ ]:
slide_elem.find("div", class_='content_title')

In [ ]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

In [ ]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

### JPL Space Images Featured Image

In [ ]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [ ]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

In [ ]:
# Use the base url to create an absolute url
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

### Mars Facts

In [ ]:
df = pd.read_html('http://space-facts.com/mars/')[0]

df.head()

In [ ]:
df.columns=['Description', 'Mars']
df.set_index('Description', inplace=True)
df

In [ ]:
df.to_html()

### Mars Weather

In [ ]:
# Visit the weather website
url = 'https://mars.nasa.gov/insight/weather/'
browser.visit(url)

In [ ]:
# Parse the data
html = browser.html
weather_soup = soup(html, 'html.parser')

In [ ]:
# Scrape the Daily Weather Report table
weather_table = weather_soup.find('table', class_='mb_table')
print(weather_table.prettify())

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [4]:
# 1. Use browser to visit the URL 
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [5]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = {}

In [6]:
html = browser.html
hemis_soup = soup(html, 'html.parser')

In [7]:
# 3. Write code to retrieve the image urls and titles for each hemisphere.
hemis = hemis_soup.find_all('div', class_='item')

title_list = []        
rel_href_list = []

for hemi in hemis:
    title = hemi.find('h3').get_text()
    link = hemi.find('a')
    href = link['href']
    img_url = f'https://astrogeology.usgs.gov{href}' 
    title_list.append(title)
    
    browser.visit(img_url)
    html2 = browser.html
    img_soup = soup(html2, 'html.parser')
    image_link = img_soup.find_all('div', class_="downloads")

    for image in image_link:
        rel_image_title = image.find('h3').get_text()
        rel_image_path = image.find('a')
        rel_image_href = rel_image_path['href']
        rel_href_list.append(rel_image_href)

#hemisphere_image_urls.update({"title": title, "img_url":rel_image_href}) 

In [8]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls = [{'title': title_list, 'img_urls': rel_href_list} for title_list,rel_href_list in zip(title_list,rel_href_list)]

hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_urls': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_urls': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_urls': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_urls': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [92]:
# 5. Quit the browser
browser.quit()